In [1]:
import os

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

In [13]:
import sounddevice as sd
from Utils import load_parameters_file, from_matrix_to_preset, NUM_PARAMETERS, MSE
import time
from Synth import Synth
import numpy as np
import cma
import sys
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS

In [19]:
def render_presets(data):
    presets, normalized = data

    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets,
        is_normalized=normalized
    )
    return synth.process_audio()

In [4]:
if __name__ == '__main__':
    target_mfcc = evaluate_presets((load_parameters_file('target.json'), False))

    x0 = np.zeros(NUM_PARAMETERS)
    sigma0 = 0.5

    # Crear estrategia
    es = cma.CMAEvolutionStrategy(x0, sigma0, {
        'popsize': 200, 
        'maxiter': 10000, 
        'bounds': [0, 1], 
        'verb_disp': 0,    # No imprimir en consola cada iteración
        'verb_log': 0,     # No generar archivos de log en disco (outcmaes...)
        'verb_time': 0,    # No mostrar estadísticas de tiempo
        'verb_filenameprefix': 'temp_' # Opcional: si genera algo, que sea con este prefijo
    })

    gen = 1

    with Pool(PROCESSORS) as pool:
        while not es.stop():
            # Generar población
            solutions = np.array(es.ask(), dtype=np.float32)  # devuelve una lista de individuos
            solutions_splitted = np.split(solutions, PROCESSORS)
            presets_splitted = [(from_matrix_to_preset(chunk), True) for chunk in solutions_splitted]
            solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
            solutions_evaluated = np.concatenate(solutions_evaluated)
            fitnesses = MSE(solutions_evaluated, target_mfcc)

            best_idx = np.argmin(fitnesses)          # índice del mejor fitness
            best_solution = solutions[best_idx]      # solución correspondiente
            best_fitness = fitnesses[best_idx]       # fitness correspondiente

            print("Gen", gen, "Mejor fitness:", best_fitness)

            es.tell(solutions, fitnesses)  # pasar fitness al algoritmo
            # es.logger.add()  # opcional, para logging interno
            # es.disp()        # imprimir estado de iteración

            gen += 1

    # Mejor solución
    best_solution = es.result.xbest
    print("Mejor individuo:", best_solution)

c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\evolution_strategy.py:1531: UserWarning: Sampling standard deviation i=0 (and 41 others) at iteration 0 multiplied by 0.6666666666666666 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={0}{4} at iteration {1}"


Gen 1 Mejor fitness: 397.00772


c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\utilities\utils.py:364: UserWarning: sigma change np.exp(1.3603801741339594) = 3.8976748153081773 clipped to np.exp(+-1) (time=Jan 10 14:18:33 2026 class=CMAAdaptSigmaCSA method=update iteration=1)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


Gen 2 Mejor fitness: 277.77924
Gen 3 Mejor fitness: 277.21158
Gen 4 Mejor fitness: 428.40698
Gen 5 Mejor fitness: 495.35297
Gen 6 Mejor fitness: 486.83093
Gen 7 Mejor fitness: 418.63705
Gen 8 Mejor fitness: 467.45953
Gen 9 Mejor fitness: 404.254
Gen 10 Mejor fitness: 404.9418
Gen 11 Mejor fitness: 442.82602
Gen 12 Mejor fitness: 431.34097
Gen 13 Mejor fitness: 363.121
Gen 14 Mejor fitness: 339.28583
Gen 15 Mejor fitness: 335.71124
Gen 16 Mejor fitness: 322.78622
Gen 17 Mejor fitness: 347.3927
Gen 18 Mejor fitness: 333.48315
Gen 19 Mejor fitness: 344.81796
Gen 20 Mejor fitness: 273.96255
Gen 21 Mejor fitness: 251.51271
Gen 22 Mejor fitness: 342.8067
Gen 23 Mejor fitness: 319.20383
Gen 24 Mejor fitness: 245.52242
Gen 25 Mejor fitness: 264.829
Gen 26 Mejor fitness: 235.23044
Gen 27 Mejor fitness: 270.5007
Gen 28 Mejor fitness: 258.01974
Gen 29 Mejor fitness: 226.61592
Gen 30 Mejor fitness: 239.4432
Gen 31 Mejor fitness: 221.07913
Gen 32 Mejor fitness: 241.89594
Gen 33 Mejor fitness: 292.1

c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\utilities\utils.py:364: UserWarning: sigma change np.exp(1.3850495030780736) = 3.99502366589014 clipped to np.exp(+-1) (time=Jan 10 14:31:38 2026 class=CMAAdaptSigmaCSA method=update iteration=351)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


Gen 352 Mejor fitness: 254.45708


c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\utilities\utils.py:364: UserWarning: sigma change np.exp(1.2639409597329656) = 3.539342444437069 clipped to np.exp(+-1) (time=Jan 10 14:31:40 2026 class=CMAAdaptSigmaCSA method=update iteration=352)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


Gen 353 Mejor fitness: 306.06445


c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\utilities\utils.py:364: UserWarning: sigma change np.exp(1.0726833992430465) = 2.9232131328020334 clipped to np.exp(+-1) (time=Jan 10 14:31:43 2026 class=CMAAdaptSigmaCSA method=update iteration=353)
  warnings.warn(msg + ' (time={}'.format(time.asctime()[4:]) +


Gen 354 Mejor fitness: 387.9085
Gen 355 Mejor fitness: 360.65295
Gen 356 Mejor fitness: 400.8936
Gen 357 Mejor fitness: 377.27078
Gen 358 Mejor fitness: 355.7171
Gen 359 Mejor fitness: 358.8115
Gen 360 Mejor fitness: 487.895
Gen 361 Mejor fitness: 427.44434
Gen 362 Mejor fitness: 551.6933
Gen 363 Mejor fitness: 483.215
Gen 364 Mejor fitness: 438.63562
Gen 365 Mejor fitness: 461.71884
Gen 366 Mejor fitness: 347.8198
Gen 367 Mejor fitness: 386.21536
Gen 368 Mejor fitness: 366.6306
Gen 369 Mejor fitness: 357.7193
Gen 370 Mejor fitness: 366.76154
Gen 371 Mejor fitness: 308.64197
Gen 372 Mejor fitness: 318.81854
Gen 373 Mejor fitness: 264.92075
Gen 374 Mejor fitness: 233.38179
Gen 375 Mejor fitness: 370.90536
Gen 376 Mejor fitness: 262.37494
Gen 377 Mejor fitness: 348.55
Gen 378 Mejor fitness: 290.4448
Gen 379 Mejor fitness: 314.1263
Gen 380 Mejor fitness: 332.64548
Gen 381 Mejor fitness: 275.345
Gen 382 Mejor fitness: 282.19263
Gen 383 Mejor fitness: 300.20694
Gen 384 Mejor fitness: 264.02

In [23]:
solution = np.expand_dims(best_solution, axis=0)
error = MSE(evaluate_presets((from_matrix_to_preset(solution), True)), target_mfcc)
print('Error', error)

target_audio = render_presets((load_parameters_file('target.json'), False))
sd.play(target_audio[0], SAMPLE_RATE)
sd.wait()

# plt.plot(target_audio[0])
# plt.show()

predicted_audio = render_presets((from_matrix_to_preset(solution), True))
sd.play(predicted_audio[0], SAMPLE_RATE)
sd.wait()

# plt.plot(predicted_audio[0])
# plt.show()

Error [22.247753]
